In [1]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [3]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows

In [4]:
site = getSiteByName("StraitsTimes")
url = site[0][2]
site_id = site[0][0]

In [5]:
t.init(visual_automation = True, chrome_browser = True)
t.url(url)

True

In [6]:
number_bb = t.count('(//div[contains(@data-vr-zone, "Top Stories")]//span[contains(@class, "story-headline")])')

df_bb = pd.DataFrame(index=range(0,number_bb-2), columns = ['Sno', 'Title', 'URL', 'Summary','Img_URL'])


for n in range(0, number_bb-2):
    title=t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]'.format(n))
    URL_b=t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]//@href'.format(n))
    URL = "https://www.straitstimes.com/" + str(URL_b)
    Img_URL = t.read('//div[contains(@data-vr-zone, "Top Stories {}")]//span[contains(@class, "story-headline")]/ancestor::div[contains(@class, "body")]/..//img/@src'.format(n))
    summaries = SummarizeUrl(URL)
    
    df_bb.iloc[n, 0] = n
    df_bb.iloc[n, 1] = title
    df_bb.iloc[n, 2] = URL
    df_bb.iloc[n, 3] = summaries
    df_bb.iloc[n, 4] = Img_URL
    
t.close()

c:\users\superhell\.conda\envs\ca1\lib\site-packages\BeautifulSoup.py:114: UserWarning: You are using a very old release of Beautiful Soup, last updated in 2011. If you installed the 'beautifulsoup' package through pip, you should know the 'beautifulsoup' package name is about to be reclaimed by a more recent version of Beautiful Soup which is incompatible with this version.

This will happen at some point after January 1, 2021.

If you just started this project, this is easy to fix. Install the 'beautifulsoup4' package instead of 'beautifulsoup' and start using Beautiful Soup 4.

If this is an existing project that depends on Beautiful Soup 3, the project maintainer (potentially you) needs to start the process of migrating to Beautiful Soup 4. This should be a relatively easy part of the Python 3 migration.

  """)


[RPA][ERROR] - cannot find //div[contains(@data-vr-zone, "Top Stories 8")]//span[contains(@class, "story-headline")]/ancestor::div[contains(@class, "body")]/..//img/@src


True

In [7]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary,img_url,latest)
              VALUES(?,?,?,?,?,?,1) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

In [12]:
def markNews(conn, site_id):
    cur = conn.cursor()
    cur.execute("UPDATE news set latest=0 where latest=1 and site_id=? ",(site_id,))
    print("News has been marked as old")

In [14]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")
markNews(conn, site_id)
for index, row in df_bb.iterrows():
    title = row['Title']
    url = row['URL']
    summary = str(row['Summary'])
    img_url = row['Img_URL']
    news = (site_id,dt_string,title,url,summary,img_url)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News has been marked as old
News 10 has been created
News 11 has been created
News 12 has been created
News 13 has been created
News 14 has been created
News 15 has been created
News 16 has been created
News 17 has been created
News 18 has been created


In [15]:
conn.close()